In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
def train_lr_model(X_train, y_train, n_cross_train):
    n_cross_train = 10
    c_range = np.arange(0.01, 0.1, 0.02)
    param_grid = [ 
        {'penalty': ['l1'], 'solver': [ 'liblinear' ], 'C': c_range, 'n_jobs': [1]},
        {'penalty': ['l2'], 'solver': [ 'liblinear' ], 'C': c_range, 'n_jobs': [1]},
        {'penalty': ['l1'], 'solver': [ 'saga'], 'C': c_range,  'n_jobs': [-1]},
        {'penalty': ['l2'], 'solver': [ 'lbfgs', 'newton-cg', 'sag'], 'C': c_range,  'n_jobs': [-1]}
    ]
    scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
    log_clf = LogisticRegression(random_state = 0, max_iter = 400)
    log_clf = GridSearchCV(log_clf, param_grid, cv = n_cross_train, refit='AUC', return_train_score = True, n_jobs = -1, scoring = scoring)
    log_clf = log_clf.fit(X_train, y_train)
    return log_clf

def test_model(classifier, X_test_df, y_tes_df):
    y_pred = classifier.predict(X_test)

    print("Global Precision:")
    print(accuracy_score(y_test, y_pred))
    print("\n")

    print("General Report:")
    print(classification_report(y_test, y_pred, target_names=['yes','no']))
    print("\n")

    print("Confusion Matrix:")
    matriz_confusion = confusion_matrix(y_test, y_pred)
    
    table = pd.DataFrame(matriz_confusion)
    table.columns = sorted(set(y_test))
    table.index = sorted(set(y_test))
    print(table)

    show_roc_cuve(y_test, y_pred)
    
def scale_data(X):
    columns = list(X)
    scaler = StandardScaler()
    scaler = scaler.fit(X)
    scaled_df = scaler.fit_transform(X)
    return pd.DataFrame(scaled_df, columns=columns)

def show_train_results(classifier, features):
    print("Best Params:", classifier.best_params_, classifier.best_score_)
    print()
    
    params = classifier.cv_results_['params']
    
    mean_train_auc = classifier.cv_results_['mean_train_AUC']
    mean_train_acc  = classifier.cv_results_['std_train_Accuracy']
    print('Train Scores')
    print_scores(mean_train_auc, mean_train_acc, params)
    
    mean_test_auc = classifier.cv_results_['mean_test_AUC']
    mean_test_acc  = classifier.cv_results_['std_test_Accuracy']
    print('Test Scores')
    print_scores(mean_test_auc, mean_test_acc, params)
     
    return [mean_test_auc, mean_test_acc]

def show_roc_cuve(y_test, y_pred):
    fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_rf, tpr_rf, label='Model')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt.show()

def print_scores(means, stds, clf_params):
    for mean, std, params in zip(means, stds, clf_params):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

In [3]:
database_folder = file_name = '../../Databases/Sinteticas'
database_path = '/breast-cancer'

In [4]:
#Load Attributes
file_name = database_folder + database_path + '/database_attr.csv'
X = pd.read_csv(file_name, sep=',')
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
#Load Gold Standard Label
file_name = database_folder + database_path + '/database_labels.csv'
labels = pd.read_csv(file_name, sep=',')
y = labels[['y']]
y.head()

,y
0,True
1,True
2,True
3,True
4,True


In [10]:
#Pre Process Data
X_scaled = scale_data(X)
X_scaled.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02
mean,-1.256562e-16,1.049736e-16,-1.272171e-16,-1.900452e-16,1.045834e-16,2.528733e-16,-1.088760e-16,-5.619407e-17,2.144350e-16,5.806720e-16,...,-7.988142e-16,-1.834112e-17,-4.015534e-16,-2.848727e-17,-2.189227e-16,-2.579464e-16,1.143393e-16,2.985310e-16,2.064351e-16,2.259470e-16
std,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,...,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00
min,-2.029648e+00,-2.229249e+00,-1.984504e+00,-1.454443e+00,-3.112085e+00,-1.610136e+00,-1.114873e+00,-1.261820e+00,-2.744117e+00,-1.819865e+00,...,-1.726901e+00,-2.223994e+00,-1.693361e+00,-1.222423e+00,-2.682695e+00,-1.443878e+00,-1.305831e+00,-1.745063e+00,-2.160960e+00,-1.601839e+00
25%,-6.893853e-01,-7.259631e-01,-6.919555e-01,-6.671955e-01,-7.109628e-01,-7.470860e-01,-7.437479e-01,-7.379438e-01,-7.032397e-01,-7.226392e-01,...,-6.749213e-01,-7.486293e-01,-6.895783e-01,-6.421359e-01,-6.912304e-01,-6.810833e-01,-7.565142e-01,-7.563999e-01,-6.418637e-01,-6.919118e-01
50%,-2.150816e-01,-1.046362e-01,-2.359800e-01,-2.951869e-01,-3.489108e-02,-2.219405e-01,-3.422399e-01,-3.977212e-01,-7.162650e-02,-1.782793e-01,...,-2.690395e-01,-4.351564e-02,-2.859802e-01,-3.411812e-01,-4.684277e-02,-2.695009e-01,-2.182321e-01,-2.234689e-01,-1.274095e-01,-2.164441e-01
75%,4.693926e-01,5.841756e-01,4.996769e-01,3.635073e-01,6.361990e-01,4.938569e-01,5.260619e-01,6.469351e-01,5.307792e-01,4.709834e-01,...,5.220158e-01,6.583411e-01,5.402790e-01,3.575891e-01,5.975448e-01,5.396688e-01,5.311411e-01,7.125100e-01,4.501382e-01,4.507624e-01
max,3.971288e+00,4.651889e+00,3.976130e+00,5.250529e+00,4.770911e+00,4.568425e+00,4.243589e+00,3.927930e+00,4.484751e+00,4.910919e+00,...,4.094189e+00,3.885905e+00,4.287337e+00,5.930172e+00,3.955374e+00,5.112877e+00,4.700669e+00,2.685877e+00,6.046041e+00,6.846856e+00


In [11]:
# Split in Train and Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state = 0, shuffle = True, stratify = y)

In [12]:
lr_model = train_lr_model(X_train, y_train, 5)
train_results = show_train_results(lr_model, X_train.columns)

Best Params: {'C': 0.08999999999999998, 'n_jobs': 1, 'penalty': 'l2', 'solver': 'liblinear'} 0.9954438860971524

Train Scores
0.973 (+/-0.012) for {'C': 0.01, 'n_jobs': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.987 (+/-0.011) for {'C': 0.03, 'n_jobs': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.991 (+/-0.009) for {'C': 0.049999999999999996, 'n_jobs': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.993 (+/-0.007) for {'C': 0.06999999999999999, 'n_jobs': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.995 (+/-0.005) for {'C': 0.08999999999999998, 'n_jobs': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.995 (+/-0.005) for {'C': 0.01, 'n_jobs': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.996 (+/-0.005) for {'C': 0.03, 'n_jobs': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.997 (+/-0.004) for {'C': 0.049999999999999996, 'n_jobs': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.997 (+/-0.005) for {'C': 0.06999999999999999, 'n_jobs': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.997 (+/-0.004) for {'

/Users/codigoriginal/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/codigoriginal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
test_model(lr_model, X_test, y_test)

Global Precision:
0.9590643274853801


General Report:
              precision    recall  f1-score   support

         yes       0.97      0.96      0.97       107
          no       0.94      0.95      0.95        64

   micro avg       0.96      0.96      0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



Confusion Matrix:


ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements